# Génération de texte avec les réseaux de neurones

Dans ce notebook, nous allons créer un réseau qui peut générer du texte, ici nous montrons que cela se fait caractère par caractère. Super post à lire ici : http://karpathy.github.io/2015/05/21/rnn-effectiveness/

Nous avons organisé le processus en "étapes" afin que vous puissiez suivre facilement avec vos propres ensembles de données.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

## Étape 1 : Les Données

Vous pouvez récupérer n'importe quel texte gratuit ici : https://www.gutenberg.org/

Nous allons choisir toutes les œuvres de Shakespeare (que nous avons déjà téléchargées pour vous), principalement pour deux raisons :

1. C'est un grand corpus de textes, il est généralement recommandé d'avoir au moins une source d'un million de caractères au total pour obtenir une génération de texte réaliste.

2. Il a un style très particulier. Comme les données textuelles utilisent un anglais ancien et sont formatées dans le style d'une pièce de théâtre, il nous apparaîtra très clairement si le modèle est capable de reproduire des résultats similaires.

In [ ]:
path_to_file = 'shakespeare.txt'

In [ ]:
text = open(path_to_file, 'r').read()

In [ ]:
print(text[:500])


                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But as the riper should by time decease,
  His tender heir might bear his memory:
  But thou contracted to thine own bright eyes,
  Feed'st thy light's flame with self-substantial fuel,
  Making a famine where abundance lies,
  Thy self thy foe, to thy sweet self too cruel:
  Thou that art now the world's fresh ornament,
  And only herald to the gaudy spring,
  Within thine own bu


### Comprendre les caractères uniques

In [ ]:
# Les caractères uniques dans le fichier
vocab = sorted(set(text))
print(vocab)
len(vocab)

['\n', ' ', '!', '"', '&', "'", '(', ')', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '>', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '|', '}']


84

## Étape 2 : Traitement du texte

### Vectorisation du texte

Nous savons qu'un réseau de neurones ne peut pas prendre en charge les données brutes des chaînes de caractères, nous devons attribuer des numéros à chaque caractère. Créons deux dictionnaires qui peuvent passer d'un index numérique à un caractère et d'un caractère à un index numérique.

In [ ]:
char_to_ind = {u:i for i, u in enumerate(vocab)}

In [ ]:
char_to_ind

{'\n': 0,
 ' ': 1,
 '!': 2,
 '"': 3,
 '&': 4,
 "'": 5,
 '(': 6,
 ')': 7,
 ',': 8,
 '-': 9,
 '.': 10,
 '0': 11,
 '1': 12,
 '2': 13,
 '3': 14,
 '4': 15,
 '5': 16,
 '6': 17,
 '7': 18,
 '8': 19,
 '9': 20,
 ':': 21,
 ';': 22,
 '<': 23,
 '>': 24,
 '?': 25,
 'A': 26,
 'B': 27,
 'C': 28,
 'D': 29,
 'E': 30,
 'F': 31,
 'G': 32,
 'H': 33,
 'I': 34,
 'J': 35,
 'K': 36,
 'L': 37,
 'M': 38,
 'N': 39,
 'O': 40,
 'P': 41,
 'Q': 42,
 'R': 43,
 'S': 44,
 'T': 45,
 'U': 46,
 'V': 47,
 'W': 48,
 'X': 49,
 'Y': 50,
 'Z': 51,
 '[': 52,
 ']': 53,
 '_': 54,
 '`': 55,
 'a': 56,
 'b': 57,
 'c': 58,
 'd': 59,
 'e': 60,
 'f': 61,
 'g': 62,
 'h': 63,
 'i': 64,
 'j': 65,
 'k': 66,
 'l': 67,
 'm': 68,
 'n': 69,
 'o': 70,
 'p': 71,
 'q': 72,
 'r': 73,
 's': 74,
 't': 75,
 'u': 76,
 'v': 77,
 'w': 78,
 'x': 79,
 'y': 80,
 'z': 81,
 '|': 82,
 '}': 83}

In [ ]:
ind_to_char = np.array(vocab)

In [ ]:
ind_to_char

array(['\n', ' ', '!', '"', '&', "'", '(', ')', ',', '-', '.', '0', '1',
       '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '>', '?',
       'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M',
       'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z',
       '[', ']', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i',
       'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v',
       'w', 'x', 'y', 'z', '|', '}'], dtype='<U1')

In [ ]:
encoded_text = np.array([char_to_ind[c] for c in text])

In [ ]:
encoded_text

array([ 0,  1,  1, ..., 30, 39, 29])

Nous disposons maintenant d'un mapping que nous pouvons utiliser pour passer des caractères aux chiffres.

In [ ]:
sample = text[:20]
sample

'\n                   '

In [ ]:
encoded_text[:20]

array([0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

## Étape 3 : Création de batches

Globalement, ce que nous essayons de faire, c'est de faire en sorte que le modèle prévoie le caractère suivant le plus probable, compte tenu d'une séquence historique de caractères. C'est à nous (l'utilisateur) de choisir la longueur de cette séquence historique. Une séquence trop courte et nous n'aurons pas assez d'informations (par exemple, étant donné la lettre "a", quel est le prochain caractère), une séquence trop longue et l'entraînement prendra trop de temps et risque de sur-entraîner (au risque d'obtenir une séquence de caractères qui ne sont pas pertinents pour des caractères plus éloignés). Bien qu'il n'y ait pas de choix correct de longueur de séquence, vous devez considérer le texte lui-même, la longueur des phrases normales qu'il contient et avoir une idée raisonnable des caractères/mots qui sont pertinents les uns pour les autres.

In [ ]:
print(text[:500])


                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But as the riper should by time decease,
  His tender heir might bear his memory:
  But thou contracted to thine own bright eyes,
  Feed'st thy light's flame with self-substantial fuel,
  Making a famine where abundance lies,
  Thy self thy foe, to thy sweet self too cruel:
  Thou that art now the world's fresh ornament,
  And only herald to the gaudy spring,
  Within thine own bu


In [ ]:
line = "From fairest creatures we desire increase"

In [ ]:
len(line)

41

In [ ]:
part_stanza = """From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But as the riper should by time decease,"""

In [ ]:
len(part_stanza)

131

### Séquences d'Entraînement

Les données textuelles réelles seront la séquence de texte décalée d'un caractère vers l'avant. 

Par exemple :

*   Sequence In: "Hello my nam"
*   Sequence Out: "ello my name"

Nous pouvons utiliser la fonction `tf.data.Dataset.from_tensor_slices` pour convertir un vecteur de texte en un flux d'indices de caractères.

In [ ]:
seq_len = 120

In [ ]:
total_num_seq = len(text)//(seq_len+1)

In [ ]:
total_num_seq

45005

In [ ]:
# Création des séquences d'entraînement
char_dataset = tf.data.Dataset.from_tensor_slices(encoded_text)

for i in char_dataset.take(500):
     print(ind_to_char[i.numpy()])



 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
1


 
 
F
r
o
m
 
f
a
i
r
e
s
t
 
c
r
e
a
t
u
r
e
s
 
w
e
 
d
e
s
i
r
e
 
i
n
c
r
e
a
s
e
,


 
 
T
h
a
t
 
t
h
e
r
e
b
y
 
b
e
a
u
t
y
'
s
 
r
o
s
e
 
m
i
g
h
t
 
n
e
v
e
r
 
d
i
e
,


 
 
B
u
t
 
a
s
 
t
h
e
 
r
i
p
e
r
 
s
h
o
u
l
d
 
b
y
 
t
i
m
e
 
d
e
c
e
a
s
e
,


 
 
H
i
s
 
t
e
n
d
e
r
 
h
e
i
r
 
m
i
g
h
t
 
b
e
a
r
 
h
i
s
 
m
e
m
o
r
y
:


 
 
B
u
t
 
t
h
o
u
 
c
o
n
t
r
a
c
t
e
d
 
t
o
 
t
h
i
n
e
 
o
w
n
 
b
r
i
g
h
t
 
e
y
e
s
,


 
 
F
e
e
d
'
s
t
 
t
h
y
 
l
i
g
h
t
'
s
 
f
l
a
m
e
 
w
i
t
h
 
s
e
l
f
-
s
u
b
s
t
a
n
t
i
a
l
 
f
u
e
l
,


 
 
M
a
k
i
n
g
 
a
 
f
a
m
i
n
e
 
w
h
e
r
e
 
a
b
u
n
d
a
n
c
e
 
l
i
e
s
,


 
 
T
h
y
 
s
e
l
f
 
t
h
y
 
f
o
e
,
 
t
o
 
t
h
y
 
s
w
e
e
t
 
s
e
l
f
 
t
o
o
 
c
r
u
e
l
:


 
 
T
h
o
u
 
t
h
a
t
 
a
r
t
 
n
o
w
 
t
h
e
 
w
o
r
l
d
'
s
 
f
r
e
s
h
 
o
r
n
a
m
e
n
t
,


 
 
A
n
d
 
o
n
l
y
 
h
e
r
a
l
d
 
t
o
 
t
h
e
 
g
a
u
d
y
 
s
p
r
i
n
g
,


 
 
W
i
t
h
i
n
 
t
h
i
n
e
 
o
w
n
 
b
u


La méthode du **batch** convertit ces appels de caractères individuels en séquences que nous pouvons alimenter en lot. Nous utilisons seq_len+1 en raison de l'indexation zéro. 

Voici ce que signifie drop_remainder :

drop_remainder: (Optional.) A `tf.bool` scalar `tf.Tensor`, representing
    whether the last batch should be dropped in the case it has fewer than
    `batch_size` elements; the default behavior is not to drop the smaller
    batch.


In [ ]:
sequences = char_dataset.batch(seq_len+1, drop_remainder=True)

Maintenant que nous avons nos séquences, nous allons effectuer les étapes suivantes pour chacune d'entre elles afin de créer nos séquences de texte cible :

1. Saisir la séquence de texte d'entrée
2. Assigner la séquence de texte cible comme séquence de texte d'entrée décalée d'un pas en avant
3. Regroupez-les en un tuple

In [ ]:
def create_seq_targets(seq):
    input_txt = seq[:-1]
    target_txt = seq[1:]
    return input_txt, target_txt

In [ ]:
dataset = sequences.map(create_seq_targets)

In [ ]:
for input_txt, target_txt in  dataset.take(1):
    print(input_txt.numpy())
    print(''.join(ind_to_char[input_txt.numpy()]))
    print('\n')
    print(target_txt.numpy())
    # Il y a un espace supplémentaire !
    print(''.join(ind_to_char[target_txt.numpy()]))

[ 0  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 12  0
  1  1 31 73 70 68  1 61 56 64 73 60 74 75  1 58 73 60 56 75 76 73 60 74
  1 78 60  1 59 60 74 64 73 60  1 64 69 58 73 60 56 74 60  8  0  1  1 45
 63 56 75  1 75 63 60 73 60 57 80  1 57 60 56 76 75 80  5 74  1 73 70 74
 60  1 68 64 62 63 75  1 69 60 77 60 73  1 59 64 60  8  0  1  1 27 76 75]

                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But


[ 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 12  0  1
  1 31 73 70 68  1 61 56 64 73 60 74 75  1 58 73 60 56 75 76 73 60 74  1
 78 60  1 59 60 74 64 73 60  1 64 69 58 73 60 56 74 60  8  0  1  1 45 63
 56 75  1 75 63 60 73 60 57 80  1 57 60 56 76 75 80  5 74  1 73 70 74 60
  1 68 64 62 63 75  1 69 60 77 60 73  1 59 64 60  8  0  1  1 27 76 75  1]
                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But 


### Générer des batches d'entraînement

Maintenant que nous avons les séquences réelles, nous allons créer les lots, nous voulons mélanger ces séquences dans un ordre aléatoire, de sorte que le modèle ne s'adapte à aucune section du texte, mais puisse au contraire générer des caractères à partir de n'importe quel texte de départ.

In [ ]:
# Taille de batch
batch_size = 128

# Taille de la mémoire tampon pour mélanger l'ensemble des données 
# afin de ne pas tenter de mélanger toute la séquence en mémoire. 
# Au lieu de cela, il maintient une mémoire tampon dans laquelle 
# il mélange les éléments
buffer_size = 10000

dataset = dataset.shuffle(buffer_size).batch(batch_size, drop_remainder=True)

In [ ]:
dataset

<BatchDataset shapes: ((128, 120), (128, 120)), types: (tf.int64, tf.int64)>

## Étape 4 : Création du modèle

Nous utiliserons un modèle basé sur le LSTM avec quelques caractéristiques supplémentaires, notamment une couche embedding pour commencer et **deux** couches LSTM. Nous avons basé cette architecture de modèle sur le [DeepMoji](https://deepmoji.mit.edu/) et le code source original peut être trouvé [ici](https://github.com/bfelbo/DeepMoji).

La couche embedding servira de couche d'entrée, qui crée essentiellement une table de consultation qui fait correspondre les indices numériques de chaque caractère à un vecteur avec un nombre de dimensions "embedding dim". Comme vous pouvez l'imaginer, plus cette taille d'embedding est grande, plus l'entraînement est complexe. C'est similaire à l'idée derrière word2vec, où les mots sont mis en correspondance avec un espace à n dimensions. L'embedding avant le feeding directe dans le LSTM conduit généralement à des résultats plus réalistes.

In [ ]:
# Longueur du vocabulaire en caractères
vocab_size = len(vocab)

# La dimension embedding
embed_dim = 64

# Nombre d'unitées RNN
rnn_neurons = 1026

Créons maintenant une fonction qui s'adapte facilement aux différentes variables comme indiqué ci-dessus.

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense,Embedding,Dropout,GRU

### Mise en place de la fonction de perte

Pour notre perte, nous utiliserons une crossentropie catégorique peu dense (sparse_categorical_crossentropy), que nous pouvons importer de Keras. Nous définirons également ce paramètre logits=True

In [ ]:
from tensorflow.keras.losses import sparse_categorical_crossentropy

In [ ]:
help(sparse_categorical_crossentropy)

Help on function sparse_categorical_crossentropy in module tensorflow.python.keras.losses:

sparse_categorical_crossentropy(y_true, y_pred, from_logits=False, axis=-1)
    Computes the sparse categorical crossentropy loss.
    
    Standalone usage:
    
    >>> y_true = [1, 2]
    >>> y_pred = [[0.05, 0.95, 0], [0.1, 0.8, 0.1]]
    >>> loss = tf.keras.losses.sparse_categorical_crossentropy(y_true, y_pred)
    >>> assert loss.shape == (2,)
    >>> loss.numpy()
    array([0.0513, 2.303], dtype=float32)
    
    Args:
      y_true: Ground truth values.
      y_pred: The predicted values.
      from_logits: Whether `y_pred` is expected to be a logits tensor. By default,
        we assume that `y_pred` encodes a probability distribution.
      axis: (Optional) Defaults to -1. The dimension along which the entropy is
        computed.
    
    Returns:
      Sparse categorical crossentropy loss value.



https://datascience.stackexchange.com/questions/41921/sparse-categorical-crossentropy-vs-categorical-crossentropy-keras-accuracy

In [ ]:
def sparse_cat_loss(y_true,y_pred):
  return sparse_categorical_crossentropy(y_true, y_pred, from_logits=True)

In [ ]:
def create_model(vocab_size, embed_dim, rnn_neurons, batch_size):
    model = Sequential()
    model.add(Embedding(vocab_size, embed_dim,batch_input_shape=[batch_size, None]))
    model.add(GRU(rnn_neurons,return_sequences=True,stateful=True,recurrent_initializer='glorot_uniform'))
    # Couche Finale Dense de Prédiction
    model.add(Dense(vocab_size))
    model.compile(optimizer='adam', loss=sparse_cat_loss) 
    return model

In [ ]:
model = create_model(
  vocab_size = vocab_size,
  embed_dim=embed_dim,
  rnn_neurons=rnn_neurons,
  batch_size=batch_size)

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (128, None, 64)           5376      
_________________________________________________________________
gru (GRU)                    (128, None, 1026)         3361176   
_________________________________________________________________
dense (Dense)                (128, None, 84)           86268     
Total params: 3,452,820
Trainable params: 3,452,820
Non-trainable params: 0
_________________________________________________________________


## Étape 5 : Entraînement du modèle

Assurons-nous que tout va bien avec notre modèle avant de passer trop de temps sur l'entraînement ! Passons en lot pour confirmer que le modèle prédit actuellement des caractères aléatoires sans aucun entraînement.



In [ ]:
for input_example_batch, target_example_batch in dataset.take(1):

  # Prédire sur un lot aléatoire
  example_batch_predictions = model(input_example_batch)

  # Afficher les dimensions des prédictions
  print(example_batch_predictions.shape, " <=== (batch_size, sequence_length, vocab_size)")


(128, 120, 84)  <=== (batch_size, sequence_length, vocab_size)


In [ ]:
example_batch_predictions

<tf.Tensor: shape=(128, 120, 84), dtype=float32, numpy=
array([[[-3.80827906e-03,  8.72645702e-04,  5.13616716e-03, ...,
          1.54101034e-03, -1.83233968e-03, -7.58333644e-03],
        [-5.28332032e-03,  1.08895488e-02,  6.62506279e-03, ...,
         -2.90334481e-03,  1.66666694e-03, -3.19830561e-03],
        [-3.33068054e-03,  2.28975131e-03,  4.56009805e-03, ...,
         -2.70648696e-03,  1.43409753e-03,  3.01969890e-03],
        ...,
        [ 3.87038430e-03,  2.44528218e-03,  2.37170211e-03, ...,
         -2.98482249e-03, -4.78985300e-03, -4.76485444e-03],
        [ 3.92475352e-03, -4.05428093e-03, -6.58480451e-04, ...,
         -2.79931747e-03, -1.76895363e-03,  1.84032950e-03],
        [-1.80049869e-03, -2.31877924e-03,  3.09810229e-03, ...,
         -8.51220742e-04,  3.15416208e-03,  6.80467067e-03]],

       [[-4.40484553e-04, -3.22631653e-03,  1.98761001e-03, ...,
         -5.31909754e-04, -1.20747136e-04,  3.51941399e-03],
        [-1.65335264e-03, -9.29711154e-04,  2.4

In [ ]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)

In [ ]:
sampled_indices

<tf.Tensor: shape=(120, 1), dtype=int64, numpy=
array([[53],
       [44],
       [46],
       [11],
       [14],
       [18],
       [50],
       [23],
       [57],
       [30],
       [35],
       [48],
       [22],
       [68],
       [16],
       [27],
       [32],
       [ 9],
       [ 5],
       [44],
       [ 4],
       [40],
       [25],
       [42],
       [ 6],
       [72],
       [48],
       [32],
       [69],
       [67],
       [ 7],
       [27],
       [10],
       [45],
       [ 7],
       [23],
       [82],
       [ 4],
       [74],
       [33],
       [58],
       [74],
       [73],
       [38],
       [60],
       [45],
       [27],
       [23],
       [11],
       [61],
       [27],
       [ 6],
       [23],
       [44],
       [27],
       [24],
       [ 2],
       [65],
       [34],
       [70],
       [38],
       [36],
       [78],
       [75],
       [ 0],
       [ 2],
       [75],
       [23],
       [10],
       [34],
       [52],
       [38],
       [64],
   

In [ ]:
# Reformater pour ne pas être une liste de listes
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [ ]:
sampled_indices

array([53, 44, 46, 11, 14, 18, 50, 23, 57, 30, 35, 48, 22, 68, 16, 27, 32,
        9,  5, 44,  4, 40, 25, 42,  6, 72, 48, 32, 69, 67,  7, 27, 10, 45,
        7, 23, 82,  4, 74, 33, 58, 74, 73, 38, 60, 45, 27, 23, 11, 61, 27,
        6, 23, 44, 27, 24,  2, 65, 34, 70, 38, 36, 78, 75,  0,  2, 75, 23,
       10, 34, 52, 38, 64, 40,  6, 51, 34,  9, 70, 22, 25, 45, 47, 53, 61,
        2, 26, 17, 83, 36, 81,  1, 73, 63, 39, 58, 44, 14, 16, 34, 39, 47,
       31, 48, 41,  6, 48, 33, 61, 29, 11, 60, 18, 63, 50, 60, 46, 28, 35,
       47])

In [ ]:
print("Compte tenu de la séquence d'entrée : \n")
print("".join(ind_to_char[input_example_batch[0]]))
print('\n')
print("Prochain caractère prédit : \n")
print("".join(ind_to_char[sampled_indices ]))


Compte tenu de la séquence d'entrée : 

ngthened though more weak in seeming,
  I love not less, though less the show appear,
  That love is merchandized, whose


Prochain caractère prédit : 

]SU037Y<bEJW;m5BG-'S&O?Q(qWGnl)B.T)<|&sHcsrMeTB<0fB(<SB>!jIoMKwt
!t<.I[MiO(ZI-o;?TV]f!A6}Kz rhNcS35INVFWP(WHfD0e7hYeUCJV


Après avoir confirmé que les dimensions fonctionnent, entraînons notre réseau !

In [ ]:
epochs = 30

In [ ]:
model.fit(dataset,epochs=epochs)

Epoch 1/30
351/351 [==============================] - 41s 117ms/step - loss: 2.5422
Epoch 2/30
351/351 [==============================] - 43s 124ms/step - loss: 1.7559
Epoch 3/30
351/351 [==============================] - 45s 129ms/step - loss: 1.4800
Epoch 4/30
351/351 [==============================] - 45s 127ms/step - loss: 1.3541
Epoch 5/30
351/351 [==============================] - 44s 127ms/step - loss: 1.2888
Epoch 6/30
351/351 [==============================] - 46s 130ms/step - loss: 1.2460
Epoch 7/30
351/351 [==============================] - 44s 125ms/step - loss: 1.2152
Epoch 8/30
351/351 [==============================] - 44s 125ms/step - loss: 1.1911
Epoch 9/30
351/351 [==============================] - 44s 127ms/step - loss: 1.1707
Epoch 10/30
351/351 [==============================] - 45s 129ms/step - loss: 1.1529
Epoch 11/30
351/351 [==============================] - 45s 127ms/step - loss: 1.1368
Epoch 12/30
351/351 [==============================] - 44s 127ms/step - lo

## Étape 6 : Génération du texte

Actuellement, notre modèle ne prévoit que 128 séquences à la fois. Nous pouvons créer un nouveau modèle qui n'attend qu'un batch_size=1. Nous pouvons créer un nouveau modèle avec cette taille de batch, puis charger les poids de nos modèles sauvegardés. Ensuite, appelez .build() sur le modèle :

In [ ]:
model.save('shakespeare_gen.h5')

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
model = create_model(vocab_size, embed_dim, rnn_neurons, batch_size=1)

model.load_weights('shakespeare_gen.h5')

model.build(tf.TensorShape([1, None]))


In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 64)             5376      
_________________________________________________________________
gru_1 (GRU)                  (1, None, 1026)           3361176   
_________________________________________________________________
dense_1 (Dense)              (1, None, 84)             86268     
Total params: 3,452,820
Trainable params: 3,452,820
Non-trainable params: 0
_________________________________________________________________


In [ ]:
def generate_text(model, start_seed,gen_size=100,temp=1.0):
  '''
  model: Modèle Entraîné pour générer du texte
  start_seed: Seed initial du texte sous forme de chaîne de caractères
  gen_size: Nombre de caractères à générer

  L'idée de base de cette fonction est de prendre un texte de départ, de le formater de manière à
  qu'il soit dans le bon format pour notre réseau, puis bouclez la séquence à mesure que
  nous continuons d'ajouter nos propres caractères prédits. Similaire à notre notre travail au sein 
  des problèmes de séries temporelles avec le RNN.
  '''

  # Nombre de caractères à générer
  num_generate = gen_size

  # Vectorisation du texte du seed de départ
  input_eval = [char_to_ind[s] for s in start_seed]

  # Étendre les dimensions pour correspondre à la forme du format de batch
  input_eval = tf.expand_dims(input_eval, 0)

  # Liste vide pour contenir le texte généré
  text_generated = []

  # La température a un effet aléatoire sur le texte qui en résulte
  # Le terme est dérivé de l'entropie/thermodynamique.
  # La température est utilisée pour affecter la probabilité des caractères suivants.
  # Probabilité plus élevée == moins surprenante/ plus attendue
  # Une température plus basse == plus surprenante / moins attendue
 
  temperature = temp

  # Ici batch size == 1
  model.reset_states()

  for i in range(num_generate):

      # Générer des prédictions
      predictions = model(input_eval)

      # Supprimer la dimension de la forme du batch
      predictions = tf.squeeze(predictions, 0)

      # Utilisez une distribution catégorielle pour sélectionner le caractère suivant
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # Passez le caractère prédit pour la prochaine entrée
      input_eval = tf.expand_dims([predicted_id], 0)

      # Transformer à nouveau en lettre de caractère
      text_generated.append(ind_to_char[predicted_id])

  return (start_seed + ''.join(text_generated))

In [ ]:
print(generate_text(model,"flower",gen_size=1000))

flower,
    In thy one from my sorrows, bid finds of these d Knock,
    Ready signior.

                TROILUS and MAYORNO find

  PETRUCHIO. Thus shall your life is on a poor thing sweet love.  
  CAMILLO. Pyille and his brow; which we may parted
    over manner.
  SERVANT. Ay, mine women.                           Exeunt




SCENE IV.
Boh. Come, Sir Thurio, fame; and that's coming forth. Pray, cherish,
    one speak for, besides to be to yakn; all sorts I see is Ca'ta throath.
  GREMIO. A wedded indume care done to Mercury,
    From whom he brought us with your royal person
    And wound thee of!  
  QUEEN ELIZABETH. Cry to your hand, 'So in himself!
  SPEED. Not a resolv'd love, sure; if you lie one of them.
    Make practise all the world more necessary.
    But hear her, let my life have life to you alone.
  PETRUCHIO. Here is a nunsion.
  SILVIA. Are you in,
    I curse the better now that titles or no; nay, witch deep precious queen.
    This sort for these unspeal, fair warran